## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

In [47]:
from numba import njit
import numpy as np
import pandas as pd

In [18]:
path1 = "D:/FinUniver/Технологии обработки больших данных/Семинары/02_pandas/data/"

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [2]:
A = np.random.randint(0, 1000, 10**6)
A

array([594, 920, 705, ..., 700, 715, 988])

In [5]:
def f1(A):
    acc, cnt = 0, 0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt += 1
    return acc/ cnt

In [4]:
%timeit f1(A)

324 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
def f2(A):
    acc, cnt = 0, 0
    for ai in A:
        acc +=ai
        cnt += 1
    acc += cnt*100
    return acc/ cnt

In [7]:
%load_ext line_profiler

In [8]:
%timeit f2(A)

230 ms ± 86.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

## Лабораторная работа 3

In [12]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` и без использования срезов таблицы

Б. С использованием метода `DataFrame.iterrows` и с использованием срезов таблицы

В. С использованием метода `DataFrame.mean`

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех вариантов.

In [19]:
reviews = pd.read_csv(path1 + "reviews_sample.csv", sep=',')
reviews

,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
5,910362,35106,31322,2003-01-03,4,I forgot to add skim milk but it still tasted ...
6,212649,404333,199579,2006-12-10,5,"Made this for dinner it was so excellent, fina..."
7,815389,162888,16067,2005-12-09,5,"When I snapped the picture, I forgot to review..."
8,642377,89831,33715,2007-07-03,5,This was good combination of flavors but I wil...
9,1023302,308434,11252,2008-12-14,5,Oh Bergy! These wonderful little cakes are aw...


In [20]:
recipes = pd.read_csv(path1 + "recipes_sample.csv", sep=',')
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
5,mennonite corn fritters,44045,15,41706,2002-10-25,NaN,ok - my heritage has been revealed. :) these a...,NaN
6,open sesame noodles,107229,28,173674,2004-12-30,8.0,this is a very versatile and widely enjoyed pa...,12.0
7,say what banana sandwich,95926,5,118163,2004-07-20,4.0,you just have to try it to believe it.,NaN
8,1 in canada chocolate chip cookies,453467,45,1848091,2011-04-11,12.0,this is the recipe that we use at my school ca...,11.0
9,412 broccoli casserole,306168,40,50969,2008-05-30,6.0,since there are already 411 recipes for brocco...,NaN


In [23]:
reviews['date'] = pd.to_datetime(reviews['date'], format="%Y-%m-%d")
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126696 entries, 0 to 126695
Data columns (total 6 columns):
Unnamed: 0    126696 non-null int64
user_id       126696 non-null int64
recipe_id     126696 non-null int64
date          126696 non-null datetime64[ns]
rating        126696 non-null int64
review        126679 non-null object
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 5.8+ MB


In [24]:
def rating_mean_1():
    s = 0
    k = 0
    for i in reviews.iterrows():
        if i[1].date.year == 2010:
            s += i[1].rating
            k += 1
    return s / k

In [25]:
def rating_mean_2():
    s = 0
    k = 0
    for i in reviews[reviews['date'].dt.year == 2010].iterrows():
        s += i[1].rating
        k += 1
    return s / k

In [26]:
def rating_mean_3():
    return reviews[reviews['date'].dt.year == 2010]['rating'].mean()

In [27]:
%time rating_mean_1()

Wall time: 11 s


4.4544402182900615

In [28]:
%time rating_mean_2()

Wall time: 1.02 s


4.4544402182900615

In [29]:
%time rating_mean_3()

Wall time: 15 ms


4.4544402182900615

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [34]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [37]:
%lprun -f rating_mean_1 rating_mean_1()

Timer unit: 1e-07 s

Total time: 30.0816 s
File: <ipython-input-24-6b378380bdea>
Function: rating_mean_1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def rating_mean_1():
     2         1         12.0     12.0      0.0      s = 0
     3         1          7.0      7.0      0.0      k = 0
     4    126697  250182672.0   1974.7     83.2      for i in reviews.iterrows():
     5    126696   46856761.0    369.8     15.6          if i[1].date.year == 2010:
     6     12094    3697024.0    305.7      1.2              s += i[1].rating
     7     12094      79274.0      6.6      0.0              k += 1
     8         1         12.0     12.0      0.0      return s / k

In [38]:
%lprun -f rating_mean_2 rating_mean_2()

Timer unit: 1e-07 s

Total time: 2.99737 s
File: <ipython-input-25-a1287ee197e6>
Function: rating_mean_2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def rating_mean_2():
     2         1         22.0     22.0      0.0      s = 0
     3         1         13.0     13.0      0.0      k = 0
     4     12095   25377671.0   2098.2     84.7      for i in reviews[reviews['date'].dt.year == 2010].iterrows():
     5     12094    4508647.0    372.8     15.0          s += i[1].rating
     6     12094      87287.0      7.2      0.3          k += 1
     7         1         12.0     12.0      0.0      return s / k

In [39]:
%lprun -f rating_mean_3 rating_mean_3()

Timer unit: 1e-07 s

Total time: 0.0187785 s
File: <ipython-input-26-9c1d08824b22>
Function: rating_mean_3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def rating_mean_3():
     2         1     187785.0 187785.0    100.0      return reviews[reviews['date'].dt.year == 2010]['rating'].mean()

3. Вам предлагается воспользоваться функцией, которая собирает информацию, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного прироста в скорости выполнения.

In [41]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [42]:
%%time
result_origin = get_word_reviews_count(reviews)

Wall time: 29.5 s


In [43]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-07 s

Total time: 86.9007 s
File: <ipython-input-41-e691285e0ae1>
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count(df):
     2         1         20.0     20.0      0.0      word_reviews = {}
     3    126680  266776544.0   2105.9     30.7      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679   58350975.0    460.6      6.7          recipe_id, review = row['recipe_id'], row['review']
     5    126679    6705196.0     52.9      0.8          words = review.split(' ')
     6   6918689   29236310.0      4.2      3.4          for word in words:
     7   6792010   42057774.0      6.2      4.8              if word not in word_reviews:
     8    174944    1233633.0      7.1      0.1                  word_reviews[word] = []
     9   6792010   44989360.0      6.6      5.2              word_reviews[word].append(recipe_id)
    10         1          9.0      9.0      0.0      word_reviews_count = {}
    11    126680  278954645.0   2202.0     32.1      for _, row in df.dropna(subset=['review']).iterrows():
    12    126679   34924909.0    275.7      4.0          review = row['review']
    13    126679    6730834.0     53.1      0.8          words = review.split(' ')
    14   6918689   32295692.0      4.7      3.7          for word in words:
    15   6792010   66751473.0      9.8      7.7              word_reviews_count[word] = len(word_reviews[word])
    16         1         11.0     11.0      0.0      return word_reviews_count

In [44]:
def get_word_reviews_count_opt(df):
    word_reviews_count = {}
    for row in df.dropna(subset=['review'])['review'].str.split(' '):
        for word in row:
            if word in word_reviews_count:
                word_reviews_count[word] += 1
            else:
                word_reviews_count[word] = 1
    return word_reviews_count

In [45]:
%%time
result_opt = get_word_reviews_count_opt(reviews)

Wall time: 2.43 s


In [46]:
%lprun -f get_word_reviews_count_opt get_word_reviews_count_opt(reviews)

Timer unit: 1e-07 s

Total time: 11.4552 s
File: <ipython-input-44-240fa494db56>
Function: get_word_reviews_count_opt at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count_opt(df):
     2         1         15.0     15.0      0.0      word_reviews_count = {}
     3    126680    9026881.0     71.3      7.9      for row in df.dropna(subset=['review'])['review'].str.split(' '):
     4   6918689   27765669.0      4.0     24.2          for word in row:
     5   6792010   37758303.0      5.6     33.0              if word in word_reviews_count:
     6   6617066   39038694.0      5.9     34.1                  word_reviews_count[word] += 1
     7                                                       else:
     8    174944     961983.0      5.5      0.8                  word_reviews_count[word] = 1
     9         1         36.0     36.0      0.0      return word_reviews_count

3. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего процентного отклонения значения рейтинга для отзыва от среднего значения рейтинга для этого отзыва.
    1. Без использования массивов `numpy` и `numba`
    2. Без использования массивов `numpy`, но с использованием `numba`
    3. С использованием массивов `numpy`, но без использования `numba`
    4. C использованием массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.

In [48]:
def MAPE_1(df: pd.DataFrame):
    rating_clear = df[df['rating'] != 0]['rating']
    mean = rating_clear.mean()
    return rating_clear.apply(lambda x: abs(mean - x)).sum() / mean * 100 / rating_clear.count()

In [49]:
%time MAPE_1(reviews)

Wall time: 161 ms


11.079150232267242

In [50]:
def MAPE_3(df: pd.DataFrame):
    rating_clear = df[df['rating'] != 0]['rating'].to_numpy()
    mean = rating_clear.mean()
    return np.absolute(rating_clear - mean).sum() / mean * 100 / rating_clear.shape[0]

In [51]:
%timeit MAPE_3(reviews)

9.19 ms ± 205 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [52]:
@njit
def help_M_4(a: np.array):
    mean = a.mean()
    return np.absolute(a - mean).sum() / mean * 100 / a.shape[0]

In [53]:
def MAPE_4(df: pd.DataFrame):
    rating_clear = df[df['rating'] != 0]['rating'].to_numpy()
    return help_M_4(rating_clear)

In [54]:
%timeit MAPE_4(reviews)

8.95 ms ± 490 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
